In [14]:
import sys
sys.path.append("/home/xiucheng/Data1/FACADES/src/")
from zensvi.download import MLYDownloader
from facades.streetview import bbox_to_geojson, process_metadata
from facades.streetview import geo_aov_calculator, filter_img_by_aov
import os
from pathlib import Path

In [2]:
mly_api_key = "MLY|7964780696888322|7d3bbcdfebb6c8f1e978c673f31e0fab" # please register your own Mapillary API key at https://www.mapillary.com/dashboard/developers
# metadata_path = f'/home/xiucheng/Data1/FACADES/test/output'
output_path = f'/home/xiucheng/Data1/FACADES/test/output'
kwarg = {"image_type": "pano"}
mly_downloader = MLYDownloader(mly_api_key)

In [3]:
bbox = [8.552,47.372,8.554,47.376]
bbox_to_geojson(bbox).to_file(os.path.join(output_path, 'bbox.geojson'), driver='GeoJSON')
# city = 'Singapore'

In [4]:
mly_downloader.download_svi(
    output_path,  # output directory
    input_shp_file=os.path.join(output_path, 'bbox.geojson'),  # path to the input shapefile containing the location information
    input_place_name="",  # name of the location to download
    resolution=2048,  # resolution of the image
    batch_size=1000,  # batch size for downloading images
    start_date=None,  # start date for downloading images (YYYY-MM-DD)
    end_date=None,  # end date for downloading images (YYYY-MM-DD)
    metadata_only=True,  # if True, only metadata is downloaded
    use_cache=True,  # if True, the cache is used
    additional_fields=["all"], # Additional fields to fetch from the API. Defaults to ["all"].
    **kwarg
)

cleaned_metadata = process_metadata(os.path.join(output_path, 'pids_urls.csv'), 
                                    os.path.join(output_path, 'pids_urls_clean.csv'))

Getting pids...
update_pids is set to False. So the following csv file will be used: /home/xiucheng/Data1/FACADES/test/output/mly_pids.csv
The panorama URLs have been read from the cache
The metadata has been downloaded
Processing chunk 1
Finished processing chunk 1


In [4]:
geo_aov_calculator(
    img_metadata=os.path.join(output_path, 'pids_urls_clean.csv'),
    json_dir=os.path.join(output_path, 'footprint.geojson'),
    buffer=30, # meter
    distance_between_points=2, # meter
    out_dir=os.path.join(output_path, 'aov.csv'),
    batch_size=20,
    max_workers=4
)

Processing batches:   0%|          | 0/13 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 13/13 [00:00<00:00, 16.54it/s]

Geometric FoV calculation completed. Results saved to: /home/xiucheng/Data1/FACADES/test/output/aov.csv


In [8]:
filter_img_by_aov(os.path.join(output_path, 'aov.csv'), 
                    os.path.join(output_path, 'pids_urls_clean.csv'),
                    max_aov=130, 
                    min_aov=10, 
                    max_image_per_building=5, 
                    method='by_aov').to_csv(os.path.join(output_path, 'pids_urls_select.csv'))

Selected 142 rows from 248 data points


In [16]:
mly_downloader.download_svi(
    output_path,
    path_pid=Path(os.path.join(output_path, 'pids_urls_select.csv')),  # if you already have a list of panorama IDs, you can specify the path to the file here
    update_pids=False,  # if True, the list of panorama IDs will be updated. If False, the list of panorama IDs will be loaded from the path_pid file
    resolution=2048,  # resolution of the image
    batch_size=1000,  # batch size for downloading images
    use_cache=True,  # if True, the cache is use
    **kwarg)

The following csv file will be used: /home/xiucheng/Data1/FACADES/test/output/pids_urls_select.csv
The panorama URLs have been read from the cache


The cache directory has been deleted


In [ ]:
# mly_downloader.download_svi(
#     output_path,
#     input_shp_file=os.path.join(output_path, 'bbox.geojson'), 
#     update_pids=False,  # if True, the list of panorama IDs will be updated. If False, the list of panorama IDs will be loaded from the path_pid file
#     resolution=2048,  # resolution of the image
#     batch_size=1000,  # batch size for downloading images
#     use_cache=True,  # if True, the cache is use
#     start_date=None,  # start date for downloading images (YYYY-MM-DD)
#     end_date=None,  # end date for downloading images (YYYY-MM-DD)
#     additional_fields=["all"],
#     **kwarg)

Getting pids...
update_pids is set to False. So the following csv file will be used: /home/xiucheng/Data1/FACADES/test/output/mly_pids.csv
The panorama URLs have been read from the cache
The cache directory has been deleted


In [ ]:
# if you want to process the metadata before downloading images, you can use the following code: 
